# Question 10

A freight plane has three large compartments to carry cargo. Weight and volume limitations of these compartments are: 

| Compartment | Weight (Tons) | Volume (m3) |
| - | - | - |
| Front | 10 | 6800 | 
| Center | 16 | 8700 |
| Rear | 8 | 5300 |

 There are four cargos waiting to be loaded in this plane. Properties of these cargos are shown on the table below: 
 
| Cargo | Cargo Total Weight (Tons) | Total Volume (m3)  | Profit (TL/ton) |
| -- | -- | -- | -- |
| K1 | 18 | 8640 | 310 |
| K2 | 15 | 9750 | 380 |
| K3 | 23 | 13340 | 350 |
| K4 | 12 | 4680 | 285 |

Furthermore, the weight of the cargo in the respective compartments must be the same proportion of that compartment's weight capacity to maintain the balance of the plane. Any proportion of these cargoes can be accepted. Formulate a linear programming model to maximize the profit by choosing how many tons of which cargo to load on the plane under these circumstances.  

In [2]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver("SCIP")
if solver is None:
    raise("SCIP solver unavailable.")

In [28]:
k1 = (18, 8640, 310)
k2 = (15, 9750, 380)
k3 = (23, 13340, 350)
k4 = (12, 4680, 285)

cargos = [k1, k2, k3, k4]
cargo_weights = [c[0] for c in cargos]
cargo_volumes = [c[1]/c[0] for c in cargos]
profits = [c[2] for c in cargos]

compartment_weights = [10, 16, 8]
compartment_volumes = [6800, 8700, 5300]

objective = solver.Objective()

result = {}
for i in range(len(compartment_weights)):
  for j in range(len(cargos)):
    n = solver.IntVar(0, cargo_weights[j], f"tons_{i}_{j}")
    result[f"compartment_{i}_cargo_{j}"] = n
    objective.SetCoefficient(n, profits[j])
  solver.Add(solver.Sum([result[f"compartment_{i}_cargo_{j}"] for j in range(len(cargos))]) <= compartment_weights[i])
  solver.Add(solver.Sum([result[f"compartment_{i}_cargo_{j}"] * cargo_volumes[j] for j in range(len(cargos))]) <= compartment_volumes[i])

objective.SetMaximization()

In [29]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()
status == pywraplp.Solver.OPTIMAL

Solving with SCIP 8.0.4 [LP solver: Glop 9.8]


True

In [30]:
print(f"Total packed value: {objective.Value()}")

Total packed value: 24580.0


In [31]:
from collections import defaultdict

compartment_final_weights = defaultdict(int)
compartment_final_volumes = defaultdict(int)

for i in range(len(compartment_weights)):
   for j in range(len(cargos)):
      key = f"compartment_{i}_cargo_{j}"
      if (n := result[key].solution_value()):
         compartment_final_weights[i] += n
         compartment_final_volumes[i] += n * cargo_volumes[j]
         print(f"{key}: {n}")

compartment_0_cargo_1: 10.0
compartment_1_cargo_1: 8.0
compartment_1_cargo_2: 2.0
compartment_1_cargo_3: 6.0
compartment_2_cargo_1: 8.0


In [32]:
compartment_final_weights, compartment_final_volumes

(defaultdict(int, {0: 10.0, 1: 16.0, 2: 8.0}),
 defaultdict(int, {0: 6500.0, 1: 8700.0, 2: 5200.0}))

In [33]:
compartment_weights, compartment_volumes

([10, 16, 8], [6800, 8700, 5300])